<a href="https://colab.research.google.com/github/DanielCob/accesible-gaussian-splatting/blob/main/accesible_gaussian_splatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Splatting - Accessible 3D Reconstruction

This notebook adapts the [original](https://github.com/graphdeco-inria/gaussian-splatting) reference implementation of "3D Gaussian Splatting for Real-Time Radiance Field Rendering" to create a simplified, educational and fully automated pipeline with a graphical interface for non-expert users.  
Developed by Daniel Cob Beirute, Instituto Tecnológico de Costa Rica (2025).

# Dependencies installation

In [5]:
# Clone and install Guassian Splatting repository
%cd /content
!git clone --recursive https://github.com/DanielCob/accesible-gaussian-splatting.git
!pip install -q plyfile

%cd /content/accesible-gaussian-splatting
!pip install -q /content/accesible-gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/accesible-gaussian-splatting/submodules/simple-knn

/content
fatal: destination path 'accesible-gaussian-splatting' already exists and is not an empty directory.
/content/accesible-gaussian-splatting
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [14]:
# Install dependencies
!sudo apt-get install -y colmap
!sudo apt-get install -y imagemagick
!sudo apt-get install -y xvfb # OpenGL workaround for Google Colab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colmap is already the newest version (3.7-2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Data upload point
(upload a short video orbiting a object or scene)

In [6]:
from google.colab import files
import os

uploaded = files.upload()
# The user uploads a single video file
uploaded_filename = list(uploaded.keys())[0]

# Create the input directory if it doesn't exist
input_dir = '/content/accesible-gaussian-splatting/data/video'
os.makedirs(input_dir, exist_ok=True)

# Define the new path for the video
video_path = os.path.join(input_dir, uploaded_filename)

# Move the uploaded video to the input directory
os.rename(uploaded_filename, video_path)

print("Video uploaded and moved to:", video_path)

Saving test8.mp4 to test8.mp4
Video uploaded and moved to: /content/accesible-gaussian-splatting/data/video/test8.mp4


# Preproccesing

In [10]:
# Create a directory to store the extracted frames
!mkdir /content/accesible-gaussian-splatting/data/input

# Use ffmpeg to extract frames from the uploaded video
!ffmpeg -i "{video_path}" -qscale:v 1 -qmin 1 -vf fps=2 /content/accesible-gaussian-splatting/data/input/%04d.jpg

print("Frames extracted to /content/gaussian-splatting/data/input/")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [11]:
%cd /content/accesible-gaussian-splatting
!python convert.py -s /content/accesible-gaussian-splatting/data/ --resize

Streaming output truncated to the last 5000 lines.
  => Merged observations: 5
  => Completed observations: 46
  => Filtered observations: 60
  => Changed observations: 0.017127

Bundle adjustment report
------------------------
    Residuals : 12074
   Parameters : 3260
   Iterations : 3
         Time : 0.043335 [s]
 Initial cost : 0.671777 [px]
   Final cost : 0.659171 [px]
  Termination : Convergence

  => Merged observations: 0
  => Completed observations: 18
  => Filtered observations: 1
  => Changed observations: 0.002941

Retriangulation

  => Completed observations: 0
  => Merged observations: 0
  => Retriangulated observations: 0

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  6.469141e+03    0.00e+00    5.78e+03   0.00e+00   0.00e+00  1.00e+04        0    1.54e-02    3.29e-02
   1  6.452534e+03    1.66e+01    4.92e+03   4.46e+00   1.00e+00  3.00e+04        1    2.83e-02    6.12e-02
  

# Training

In [12]:
%cd /content/accesible-gaussian-splatting
!python train.py -s /content/accesible-gaussian-splatting/data/

[Errno 2] No such file or directory: '/content/gaussian-splatting'
/content/accesible-gaussian-splatting
2025-10-29 14:02:12.156766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761746532.189366    8395 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761746532.199203    8395 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761746532.224299    8395 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761746532.224335    8395 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same targe

# Exporting output
(.zip can later be visualized using other visualization tools like the one provided by [Camenduru](https://colab.research.google.com/github/camenduru/gaussian-splatting-colab/blob/main/gaussian_splatting_viewer_colab.ipynb) )

In [13]:
import shutil
import os
from google.colab import files

output_dir = '/content/accesible-gaussian-splatting/output'
compressed_output_path = '/content/accesible-gaussian-splatting/output.zip'

# Compress the output folder
shutil.make_archive(compressed_output_path.replace('.zip', ''), 'zip', output_dir)

# Provide a download link
files.download(compressed_output_path)

print(f"Output folder compressed to {compressed_output_path} and ready for download.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Output folder compressed to /content/accesible-gaussian-splatting/output.zip and ready for download.
